<a href="https://colab.research.google.com/github/BEKKAMSHIVA/OCR_TEXT_EXTRACTION/blob/main/OCR_Adharra__Card.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Environment Setup

In [1]:
 #These libraries are used to set up the environment for extracting text from images using OCR.
!pip install pytesseract opencv-python
!sudo apt-get install -y tesseract-ocr

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


#Upload and Validate .jpg Files

In [2]:
#Importing Required Libraries
import cv2
import pytesseract
from google.colab import files

In [ ]:
# only jpg format files need to uploaded here
from google.colab import files
uploaded = files.upload()
if len(uploaded) == 0:
    print("No addhar card is not uploaded and Skipping this process.")
else:
    file_name = next(iter(uploaded))
    if not file_name.lower().endswith('.jpg'):
        print("Error: Please upload the addhar card in the jpg format only.")
    else:
        print(f"File '{file_name}' addhar card uploaded successfully.")



#upload the jpg file from your system directly(choose files).

# Displaying  the Uploaded Image for Cross-Checking

In [ ]:
#Cross checking whether we have uploaded correct jpg file or not.
from IPython.display import Image, display
display(Image(file_name, width=200))

# Image Reading and  Grayscale convetion Preprocess.

In [ ]:
#Converting in to Grayscale and Thresholding to Enhance Image Clarity for Better Text Detection.
image = cv2.imread(file_name)
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)                                         # Coverting in to grayscale
gray = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]             # Applying Thresholding its  optional but improves accuracy



import matplotlib.pyplot as plt       #Display the Grayscale Image
plt.imshow(gray, cmap='gray')
plt.axis('off')
plt.title(' Grayscale Preprocessed Image')
plt.show()


#Text Extraction using OCR

In [ ]:
# Moving the downloaded Telugu language file to the tessdata folder
# Without this step, Tesseract will not detect Telugu text.
!mv /content/tel.traineddata /usr/share/tesseract-ocr/4.00/tessdata/


mv: cannot stat '/content/tel.traineddata': No such file or directory


In [ ]:
# Listing all the languages currently available in Tesseract OCR.
# If 'tel' appears in the list, Telugu text extraction can proceed.
!tesseract --list-langs

List of available languages (3):
eng
osd
tel


In [ ]:
# Extract English and Telugu Text using OCR    #by using the oem 3 --psm 6 we can imporve the  accuracy and efficiency of text extraction.
extracted_text = pytesseract.image_to_string(gray, lang='eng+tel', config='--oem 3 --psm 6')

#Drawing the bounding boxes around the extracted text.

In [ ]:
#Drawing the bounded box around what text we extracted form the given jpg file
details = pytesseract.image_to_data(image, output_type=pytesseract.Output.DICT, lang='eng+tel', config='--oem 3 --psm 6')
n_boxes = len(details['level'])                                                                          # Draw bounding boxes
for i in range(n_boxes):
    (x, y, w, h) = (details['left'][i], details['top'][i], details['width'][i], details['height'][i])
    if int(details['conf'][i]) > 30:                                                                     # Confidence threshold
        cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)

In [ ]:
# Resized  the image to 500 pixels width before displaying
target_width = 500
height, width = image.shape[:2]
aspect_ratio = target_width / width
new_dimensions = (target_width, int(height * aspect_ratio))
resized_image = cv2.resize(image, new_dimensions, interpolation=cv2.INTER_AREA)

# Displaying  the resized image with bounding boxes.
from google.colab.patches import cv2_imshow
cv2_imshow(resized_image)


#Display Output

In [ ]:
#Display Extracted Text
print("Extracted Text from the Uploaded Image:")
print(extracted_text)

#Saving the extracted text

In [ ]:
# saving the extracted text from the jpg image
file_name = "Extracted_Text_Addhar_Eng_Tel.txt"
with open(file_name, "w") as file:
   file.write(extracted_text)

print(f" Text successfully saved to '{file_name}'")

#Create a download link for the file

In [ ]:
#Create a download link for the extracted text for our access and further analysis in to our system
from google.colab import files
files.download(file_name)
